# ODE Integrators II: Implicit and Symplectic Methods

## Convergence of Integrators

When solving ordinary differential equations (ODEs) numerically,
errors inevitably enter through truncation (e.g., approximating
derivatives with finite differences) and round-off (finite precision
arithmetic).
If these errors grow uncontrollably, the computed solution may diverge
from the true behavior, even if the method is very accurate for short
times.

It is useful to distinguish between several related ideas:
* Accuracy:
  How close the numerical solution is to the exact solution at a given
  time step.
* Consistency:
  The numerical method reproduces the original ODE as
  $\Delta t \to 0$.
  Formally, the local truncation error (the error made in a single
  step assuming exact input) should vanish as $\Delta t \to 0$.
* Stability:
  Errors introduced during computation do not grow uncontrollably as
  steps are repeated.
* Convergence:
  The global numerical solution approaches the exact solution as
  $\Delta t \to 0$.

These concepts are related by a central result of numerical analysis:
\begin{align}
  \text{Consistency + Stability } \implies \text{ Convergence.}
\end{align}

This is known as the
[Lax Equivalence Theorem](https://en.wikipedia.org/wiki/Lax_equivalence_theorem).
The theorem is really for linear finite difference methods for partial
different equation, but it is still useful to discuss it in ODE
integrator.
It gives us a practical recipe:
* First check that the method is consistent (usually straightforward).
* Then analyze stability, we usually use the linear test equation:
  \begin{align}
    \frac{dx}{dt} = \lambda x, \quad \lambda \in \mathbb{C}.
  \end{align}
  Its exact solution is $x(t) = x_0 e^{\lambda t}$.
  Applying a numerical method produces an update:
  \begin{align}
    x_{n+1} = R(z) x_n, \quad z = \lambda \Delta t,
  \end{align}
  where $R(z)$ is the amplification factor.
  A method is stable if
  \begin{align}
    |R(z)| \leq 1,
  \end{align}
  so that errors do not amplify step by step.
  The set of $z$ satisfying this condition defines the stability region.
* Convergence then follows automatically.

### Example: Forward Euler Consistency

The Forward Euler method for $dx/dt = f(x)$ is
\begin{align}
  x_{n+1} = x_n + \Delta t f(x_n).
\end{align}
Expanding the true solution with a Taylor series:
\begin{align}
  x(t_{n+1}) = x(t_n + \Delta t) = x(t_n) + \Delta t f(x_n) + \frac{1}{2} \Delta t^2 f'(x_n) + \mathcal{O}(\Delta t^3)
\end{align}

The difference between the true solution and the Forward Euler step is
$\mathcal{O}(\Delta t^2)$.
Thus, the local truncation error is $\mathcal{O}(\Delta t^2)$, meaning
the method is first-order consistent.

Applying the test equation to forward Euler method gives:
\begin{align}
  x_{n+1} = (1 + \lambda \Delta t) x_n,
\end{align}
so the amplification factor is
\begin{align}
  R(z) = 1 + z, \quad z = \lambda \Delta t.
\end{align}

The stability condition requires
\begin{align}
  |1 + z| \leq 1,
\end{align}
which is the interior of the circle centered at $(-1,0)$ with radius 1
in the complex plane.

In [ ]:
import numpy as np

# Define grid in complex plane
Re = np.linspace(-3, 3, 601)
Im = np.linspace(-3, 3, 601)

Re, Im = np.meshgrid(Re, Im)
Z = Re + 1j * Im

In [ ]:
# Forward Euler amplification factor

R = abs(1 + Z)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

plt.contour (Re, Im, R, levels=[1],   colors=['C0'], linewidths=2)
plt.contourf(Re, Im, R, levels=[0,1], colors=['C0'], alpha=0.1)

plt.legend(handles=[
    mpatches.Patch(color='C0', label="Forward Euler Stable Region"),
])

plt.title('Stability Region: Forward Euler')
plt.xlabel(r'Re($z$) = Re($\lambda\Delta t$)')
plt.ylabel(r'Re($z$) = Im($\lambda\Delta t$)')
plt.gca().set_aspect('equal')

Combined with stability, forward Euler is first-order convergent,
i.e., the global error scales as $\mathcal{O}(\Delta t)$, within its
stable region.
Outside the stable region, the method diverges.